# Arxiv Explorer Tools
- extract articles on topics of interest from the too-many-to-look-through loads of articles that come out each day.
- adjust how strict of loose your filters are
- save results to json and html
- gguf embedder should run on any cpu or hardware

### Setup & Install:
- Make sure you have c-tools installed for python-llama-cpp to run
- have python installed and use an python env
- use a jupyter notebook or script, etc.

### Note, the current version is not yet able to grab a link or article id
work in progresss...
  

## currently uses this embedding model:

https://huggingface.co/CompendiumLabs/bge-large-en-v1.5-gguf

But mostly any gguf embedder model should work in this system,
and any running embedder model can be added into this system (e.g. if you have a good working local gpu setup). 

- https://pypi.org/project/llama-cpp-python/
- https://pypi.org/project/scikit-learn/
- https://pypi.org/project/beautifulsoup4/

requirements.txt ->
```
llama-cpp-python
scikit-learn 
scipy 
numpy
beautifulsoup4
```

In [13]:
# !pip install -q llama-cpp-python
# !pip install -q scikit-learn scipy numpy
# !pip install -q beautifulsoup4

In [14]:
!ls

archive
articles2024-06-08__221114049623.html
articles_2024-06-08__221114049623.json
articles2024-06-08__224351357227.html
articles_2024-06-08__224351357227.json
articles2024-06-08__230035837120.html
articles_2024-06-08__230035837120.json
bge-large-en-v1.5-f32.gguf
bge-large-en-v1.5-q4_k_m.gguf
embedding_model_llama_cpp_search_match_v10.ipynb
embedding_model_llama_cpp_search_match_v11.ipynb
embedding_model_llama_cpp_search_match_v12.ipynb
embedding_model_llama_cpp_search_match_v13.ipynb
embedding_model_llama_cpp_search_match_v14.ipynb
embedding_model_llama_cpp_search_match_v9.ipynb
embedding_model_llama_cpp_test.ipynb
env
requirements.txt


In [15]:
model_path = "bge-large-en-v1.5-f32.gguf"

In [16]:
text_to_embed = "hello world"

In [17]:
from llama_cpp import Llama
model = Llama(model_path, embedding=True)

llama_model_loader: loaded meta data with 23 key-value pairs and 389 tensors from bge-large-en-v1.5-f32.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = bge-large-en-v1.5
llama_model_loader: - kv   2:                           bert.block_count u32              = 24
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 1024
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 4096
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 16
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f32   

In [18]:
embed = model.embed(text_to_embed)


llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     121.27 ms /     4 tokens (   30.32 ms per token,    32.98 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     124.34 ms /     5 tokens


In [19]:
## inspection to see if pipeline is working
# print(embed)

In [20]:
embedding1 = model.embed("flower")
embedding2 = model.embed("blossom")


llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      64.46 ms /     3 tokens (   21.49 ms per token,    46.54 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      65.80 ms /     4 tokens

llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      50.75 ms /     3 tokens (   16.92 ms per token,    59.12 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      53.35 ms /     4 tokens


In [21]:
#############################################
# get_vector function to modularize pipeline
#############################################
def get_vector(text_to_vectorize):
    return model.embed(text_to_vectorize)

In [22]:
# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text, else: move on

In [23]:
#############
# Functions
############

from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_distance(embedding1, embedding2, boolean=False, threshold=0.6):
    """
    Cosine Similarity: This is a common method for measuring the similarity
    between two vectors. It measures the cosine of the angle between
    two vectors and the result is a value between -1 and 1.
    A value of 1 means the vectors are identical,
    0 means they are orthogonal (or completely dissimilar),
    and -1 means they are diametrically opposed.

    if not surprisingly, this looks solid: gold standard?
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = cosine_similarity([embedding1], [embedding2])

    similarity = similarity[0][0]
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import euclidean

def euclidean_distance(embedding1, embedding2, boolean=False, threshold=0.5):
    """
    Euclidean Distance: This is another common method for measuring
     the similarity between two vectors.
     It calculates the straight-line distance between two points in a space.
     The smaller the distance, the more similar the vectors.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + euclidean(embedding1, embedding2))
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



import numpy as np

def normalized_dot_product(embedding1, embedding2, boolean=False, threshold=0.6):
    """
    Dot Product: This is a simple method that calculates
    the sum of the products of the corresponding entries of the
    two sequences of numbers. If the vectors are normalized,
    the dot product is equal to the cosine similarity.

    0.5 ok? seems good
    """
    # Assuming embedding1 and embedding2 are your embeddings
    dot_product = np.dot(embedding1, embedding2)
    normalized_dot_product = dot_product / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

    similarity = normalized_dot_product
    
    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import cityblock

def manhattan_distance(embedding1, embedding2, boolean=False, threshold=0.0024):
    """
    Manhattan Distance: This is a measure of the distance between
    two vectors in a grid-based system.
    It calculates the sum of the absolute differences of their coordinates.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + cityblock(embedding1, embedding2))


    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



from scipy.stats import pearsonr

def pearson_correlation(embedding1, embedding2, boolean=False, threshold=0.6):
    """
    Pearson Correlation: This is a measure of the linear correlation
    between two vectors. It ranges from -1 (perfectly negatively correlated)
     to 1 (perfectly positively correlated).

    maybe decent around 0.6?
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = pearsonr(embedding1, embedding2)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



from scipy.stats import spearmanr

def spearmans_rank_correlation(embedding1, embedding2, boolean=False, threshold=0.6):
    """
    Spearman's Rank Correlation: This is a non-parametric
     measure of the monotonicity of the relationship between
     two datasets. Unlike the Pearson correlation, the Spearman
      correlation does not assume that the relationship between
       the two variables is linear.

    more strict measure?
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = spearmanr(embedding1, embedding2)


    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True


from scipy.stats import kendalltau
def kendalls_rank_correlation(embedding1, embedding2, boolean=False, threshold=0.7):
    
    """
    Kendall's Rank Correlation: This is another non-parametric
    measure of the ordinal association between two variables.
    It is a measure of the correspondence between two rankings.

    0.3 may match the subject generally
    0.5 may most closely match meaning
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity, _ = kendalltau(embedding1, embedding2)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import minkowski


def minkowski_distance(embedding1, embedding2, boolean=False, threshold=0.055):
    """
    Minkowski Distance: This is a generalization of
    both the Euclidean distance and the Manhattan distance.
    It is defined as the p-th root of the sum of the p-th powers
    of the differences of the coordinates.
    When p=1, this is the Manhattan distance,
    and when p=2, this is the Euclidean distance.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + minkowski(embedding1, embedding2, p=2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import chebyshev
def chebyshev_distance(embedding1, embedding2, boolean=False, threshold=0.4):
    """
    Chebyshev Distance: This is a measure of the distance between
    two vectors in a vector space.
    It is the maximum of the absolute differences of their coordinates.
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + chebyshev(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



import numpy as np
from scipy.spatial.distance import mahalanobis
from numpy.linalg import inv

def mahalanobis_distance(embedding1, embedding2, boolean=False, threshold=0.415):
    """Mahalanobis Distance: This is a measure of the distance between 
    a point P and a distribution D, introduced by P. C. Mahalanobis in 1936.
    It is a multivariate generalization of the Euclidean distance.
    It is based on correlations between dimensions of the data, 
    and thus takes into account the structure of the data.
    """

    # Assuming embedding1 and embedding2 are your vectors
    data = np.array([embedding1, embedding2])

    # Calculate the covariance matrix with a small regularization term
    cov = np.cov(data, rowvar=False) + np.eye(data.shape[1])# * 1e-6

    # Calculate the Mahalanobis distance
    distance = mahalanobis(embedding1, embedding2, inv(cov))

    # Calculate the similarity score
    similarity = 1 / (1 + distance)

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True




from scipy.spatial.distance import braycurtis
def bray_curtis_distance_dissimilarity(embedding1, embedding2, boolean=False, threshold=0.75):
    """Bray-Curtis Distance: This is a measure of dissimilarity
    between two vectors. It is used in ecology to compare species
    composition in different samples. It is defined as the sum of
    the absolute differences between the vectors, divided by the sum of their sums.

    0.75 is maybe a stricker-yes

    but total no is still .6+
    """
    
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + braycurtis(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



from scipy.spatial.distance import canberra
def canberra_distance(embedding1, embedding2, boolean=False, threshold=0.002):
    """
    dissimilarity
    Canberra Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as the sum of the absolute
    differences between the vectors, divided by the sum of their absolute values.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + canberra(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True




from scipy.stats import pearsonr
def correlation_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.7):
    """
    dissimilarity
    Correlation Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as 1 - the absolute value of
    the Pearson correlation coefficient between the vectors.

    even no is hight... maybe .7 ok?
    """
    # Assuming embedding1 and embedding2 are your embeddings
    correlation, _ = pearsonr(embedding1, embedding2)
    similarity = 1 / (1 + (1 - abs(correlation)))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True




from scipy.spatial.distance import sqeuclidean
def squared_euclidean_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.005):
    """
    dissimilarity
    Squared Euclidean Distance: This is a measure of the dissimilarity
    between two vectors. It is defined as the sum of the squared differences
    between the vectors. It is similar to the Euclidean distance,
    but it does not take the square root, which can make it faster to compute.
    """
    # Assuming embedding1 and embedding2 are your embeddings
    similarity = 1 / (1 + sqeuclidean(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True


from scipy.spatial.distance import hamming
def hamming_distance_dissimilarity_measure(embedding1, embedding2):
    """
    Hamming Distance: This is a measure of the minimum number
    of substitutions required to change one vector into the other.
    It is used in information theory to measure the difference between
    two binary vectors.
    """
    # Assuming embedding1 and embedding2 are your binary vectors
    similarity = 1 / (1 + hamming(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True



# """
# not for vectors
# dissimilarity
# Jensen-Shannon Distance: This is a measure of the dissimilarity
# between two probability distributions. It is defined as the square root
# of the Jensen-Shannon divergence. It is a symmetric and smooth measure
# of dissimilarity that is always greater than or equal to 0.
# """
# from scipy.spatial.distance import jensenshannon
# import numpy as np
# def jensen_shannon_distance_dissimilarity_measure(embedding1, embedding2):
#     # Assuming embedding1 and embedding2 are your probability distributions
#     # Compute the average of the two distributions
#     average = 0.5 * np.add(embedding1, embedding2)
#     # Compute the Jensen-Shannon divergence
#     jsd = 0.5 * jensenshannon(embedding1, average) + 0.5 * jensenshannon(embedding2, average)
#     # Compute the Jensen-Shannon distance
#     dissimilarity = np.sqrt(jsd)
#     return dissimilarity



# """
# not for vectors
# dissimilarity
# Kullback-Leibler Divergence:
# This is a measure of the dissimilarity between two probability distributions.
#  It is not symmetric, meaning that the divergence from
#  distribution P to distribution Q is not
#  necessarily the same as the divergence from Q to P.
# """
# from scipy.special import kl_div
# def kullback_leibler_distance_dissimilarity_measure(embedding1, embedding2):
#     # Assuming embedding1 and embedding2 are your probability distributions
#     # Compute the Kullback-Leibler divergence
#     kld = np.sum(kl_div(embedding1, embedding2))
#     return kld



from scipy.stats import wasserstein_distance
def total_variation_distance_dissimilarity_measure(embedding1, embedding2, boolean=False, threshold=0.97):
    """
    dissimilarity
    Total Variation Distance: This is a measure of the dissimilarity
    between two probability distributions.
    It is defined as half the sum of the absolute differences
    between the corresponding probabilities in the two distributions.

    all scores high, maybe .97 is strict enough?
    """
    # Assuming embedding1 and embedding2 are your probability distributions
    similarity = 1 / (1 + wasserstein_distance(embedding1, embedding2))

    if not boolean:
        # print(similarity)
        return similarity

    else:
        if similarity < threshold:
            return False

        else:
            return True


In [24]:
###################
# Arxiv Explorerer
###################

# step 1: embed the search-phrase
# step 2: embed each text
# step 3: get scores
# step 4: evaluates if score is succss or fail
# step 5: if success: do stuff with text



import requests
from bs4 import BeautifulSoup
import json

##########################################
# Make comparison phrase and vectorize it
##########################################
comparison_phrase = "computer vision resolution enhancement"
# comparison_phrase = "cyber security"
# comparison_phrase = "natural language processing"


embedding1 = get_vector(comparison_phrase)


# Make a request to the website
r = requests.get('https://arxiv.org/list/cs/new')
soup = BeautifulSoup(r.text, 'html.parser')

# Find all the articles
articles = soup.find_all('div', class_='meta')

# List to hold all article data
article_data = []

for article in articles:
    # Extract the title
    title = article.find('div', class_='list-title mathjax').text.strip().replace('Title:', '')

    # Extract the abstract
    abstract = article.find('p', class_='mathjax').text.strip()

    # Extract the link
    title_div = article.find('div', class_='list-title mathjax')
    if title_div:
        link_element = title_div.find('a')
        if link_element:
            link = 'https://arxiv.org' + link_element['href']
        else:
            link = ''
    else:
        link = ''


    # Extract the subjects
    subjects_element = article.find('div', class_='list-subjects')
    if subjects_element:
        subjects = subjects_element.text.strip().replace('Subjects:', '').split(';')
        subjects = [subject.strip() for subject in subjects]
    else:
        subjects = []


    extracted_article_string = title + " " + abstract


    ############################
    # Do embedding search here:
    ############################

    embedding2 = get_vector(extracted_article_string)

    
    ##################################
    # Do basic embedding search here:
    ##################################

    list_of_comparison_function_tuples = [
        (cosine_similarity_distance, "cosine_similarity_distance"),
        (correlation_distance_dissimilarity_measure, "correlation_distance_dissimilarity_measure"),
        (pearson_correlation, "pearson_correlation"),
        (canberra_distance, "canberra_distance"),
        (euclidean_distance, "euclidean_distance"),
        (manhattan_distance, "manhattan_distance"),
        (minkowski_distance, "minkowski_distance"),
        (squared_euclidean_distance_dissimilarity_measure, "squared_euclidean_distance_dissimilarity_measure"),
        (chebyshev_distance, "chebyshev_distance"),
        (kendalls_rank_correlation, "kendalls_rank_correlation"),
        (bray_curtis_distance_dissimilarity, "bray_curtis_distance_dissimilarity"),
        (normalized_dot_product, "normalized_dot_product"),
        (spearmans_rank_correlation, "spearmans_rank_correlation"),
        (total_variation_distance_dissimilarity_measure, "total_variation_distance_dissimilarity_measure"),
    ]


    # Arguments to pass to the functions
    arguments = (embedding1, embedding2, True)

    # print(f"For {comparison_phrase} vs. {extracted_article_string}")

    list_of_boolean_scores = []
    
    # Iterate through the functions and call each one with the arguments
    for this_function_tuple in list_of_comparison_function_tuples:
        function_pointer = this_function_tuple[0]
        raw_score = function_pointer(*arguments)

        # print(raw_score)
        list_of_boolean_scores.append(raw_score)

    
    ratio_score = list_of_boolean_scores.count(True)

    print(f"{ratio_score} / {len(list_of_boolean_scores)}")

    # input("PointBreak")

    decimal_percent_true = ratio_score / len(list_of_boolean_scores)
    
    # target_score_decimal_percent = 0.5
    target_score_decimal_percent = 5 / len(list_of_boolean_scores)

    # Check if the abstract contains any of the keywords
    if decimal_percent_true >= target_score_decimal_percent:

        # Append the data to the list
        article_data.append({
            'title': title,
            'abstract': abstract,
            'link': link,
            'subjects': subjects,
        })


#############
# Write Data
#############

# Posix UTC Seconds
# make readable time
from datetime import datetime, UTC
date_time = datetime.now(UTC)
clean_timestamp = date_time.strftime('%Y-%m-%d__%H%M%S%f')


# Save the data to a JSON file
with open(f'articles_{clean_timestamp}.json', 'w') as f:
    json.dump(article_data, f)


# Create an HTML file
html = '<html><body>'
for article in article_data:
    html += f'<h2><a href="{article["link"]}">{article["title"]}</a></h2>'
    html += f'<p>{article["link"]}</p>'
    html += f'<p>{article["abstract"]}</p>'
    html += f'<p>Subjects: {", ".join(article["subjects"])}</p>'

html += '</body></html>'


# Save the HTML to a file
with open(f'articles{clean_timestamp}.html', 'w') as f:
    f.write(html)




llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     134.24 ms /     6 tokens (   22.37 ms per token,    44.70 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     136.29 ms /     7 tokens

llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1030.34 ms /   331 tokens (    3.11 ms per token,   321.25 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1031.87 ms /   332 tokens


0 / 14



llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     762.21 ms /   238 tokens (    3.20 ms per token,   312.25 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     764.77 ms /   239 tokens


0 / 14



llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     599.50 ms /   225 tokens (    2.66 ms per token,   375.31 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     601.50 ms /   226 tokens


...

0 / 14



llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     390.63 ms /   153 tokens (    2.55 ms per token,   391.68 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     392.91 ms /   154 tokens


0 / 14



llama_print_timings:        load time =     124.28 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     766.59 ms /   294 tokens (    2.61 ms per token,   383.52 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     769.80 ms /   295 tokens


0 / 14
